# OpenAI コンプライアンスログプラットフォーム クイックスタート

このノートブックを使用して、OpenAI コンプライアンスログプラットフォームの使用を開始してください。例では、ログファイルをダウンロードしてSIEMやデータレイクに取り込めるようにすることに焦点を当てています。

- [ヘルプセンター概要](https://help.openai.com/en/articles/9261474-compliance-api-for-chatgpt-enterprise-edu-and-chatgpt-for-teachers)
- [API リファレンス](https://chatgpt.com/admin/api-reference#tag/Compliance-API-Logs-Platform)

## 前提条件

- `COMPLIANCE_API_KEY`としてエクスポートされたEnterprise Compliance APIキー
- 対象となるプリンシパルのChatGPTアカウントIDまたはAPI Platform組織ID
- あなたの環境に固有の要件

## クイックスタートスクリプト

以下に、機能的に同一のスクリプトを提供します - Unix系環境用とWindows系環境用の2つです。
これらのスクリプトは、Compliance APIとの統合を構築して、指定されたイベントタイプと時間範囲のログデータを取得・処理する方法の例を示しています。
これらのスクリプトは、利用可能なログファイルの一覧表示とページング処理、およびそれらのダウンロードを処理し、出力をstdoutに書き込みます。

これらのスクリプトの実行例は、ヘルプブロック内に埋め込まれています - 引数なしで実行すると確認できます。

## オプション1: Unix系

前提条件：
- スクリプトを`download_compliance_files.sh`としてローカルに保存し、実行可能にマークする
- 最新の`bash`、`curl`、`sed`、`date`がインストールされていることを確認する
- すべてのログを取得したい日付の`after`パラメータを、タイムゾーンを含むISO 8601文字列形式でフォーマットする

スクリプトを`./download_compliance_files.sh <workspace_or_org_id> <event_type> <limit> <after>`のように実行する

```bash
#!/usr/bin/env bash
set -euo pipefail

usage() {
  echo "使用方法: $0 <workspace_or_org_id> <event_type> <limit> <after>" >&2
  echo >&2
  echo '例: ' >&2
  echo 'COMPLIANCE_API_KEY=<KEY> ./download_compliance_files.sh f7f33107-5fb9-4ee1-8922-3eae76b5b5a0 AUTH_LOG 100 "$(date -u -v-1d +%Y-%m-%dT%H:%M:%SZ)" > output.jsonl' >&2
  echo 'COMPLIANCE_API_KEY=<KEY> ./download_compliance_files.sh org-p13k3klgno5cqxbf0q8hpgrk AUTH_LOG 100 "$(date -u -v-1d +%Y-%m-%dT%H:%M:%SZ)" > output.jsonl' >&2
}

if [[ $# -ne 4 ]]; then
  usage
  exit 2
fi

PRINCIPAL_ID="$1"
EVENT_TYPE="$2"
LIMIT="$3"
INITIAL_AFTER="$4"

# COMPLIANCE_API_KEYが存在し、空でないことを確認してから使用する
if [[ -z "${COMPLIANCE_API_KEY:-}" ]]; then
  echo "COMPLIANCE_API_KEY環境変数が必要です。例:" >&2
  echo "COMPLIANCE_API_KEY=<KEY> $0 <workspace_or_org_id> <event_type> <limit> <after>" >&2
  exit 2
fi

API_BASE="https://api.chatgpt.com/v1/compliance"
AUTH_HEADER=("-H" "Authorization: Bearer ${COMPLIANCE_API_KEY}")

# 最初の引数がワークスペースIDか組織IDかを判断する
# "org-"で始まる場合は組織IDとして扱い、パスセグメントを適切に切り替える
SCOPE_SEGMENT="workspaces"
if [[ "${PRINCIPAL_ID}" == org-* ]]; then
  SCOPE_SEGMENT="organizations"
fi

# curlリクエストを実行し、HTTPエラー時は高速で失敗し、コンテキストをstderrにログ出力する
# 使用方法: perform_curl "アクションの説明" <curl引数...>
perform_curl() {
  local description="$1"
  shift
  # ボディとHTTPステータスコードを取得し、ボディをstdoutライクな変数に保持
  # ボディに末尾の改行がない場合でも確実に分割できるよう、ステータスの前に改行を追加
  local combined
  if ! combined=$(curl -sS -w "\n%{http_code}" "$@"); then
    echo "${description}中にネットワーク/トランスポートエラーが発生しました" >&2
    exit 1
  fi
  local http_code
  http_code="${combined##*$'\n'}"
  local body
  body="${combined%$'\n'*}"

  if [[ ! "${http_code}" =~ ^2[0-9][0-9]$ ]]; then
    echo "${description}中にHTTPエラー ${http_code} が発生しました:" >&2
    if [[ -n "${body}" ]]; then
      # stdoutストリームを破損させないよう、ボディをstderrに出力
      echo "${body}" | jq . >&2
    fi
    exit 1
  fi

  # 成功時は、呼び出し元が使用できるようボディをstdoutに出力
  echo "${body}"
}

list_logs() {
  local after="$1"
  perform_curl "ログの一覧取得 (after=${after}, event_type=${EVENT_TYPE}, limit=${LIMIT})" \
    -G \
    "${API_BASE}/${SCOPE_SEGMENT}/${PRINCIPAL_ID}/logs" \
    "${AUTH_HEADER[@]}" \
    --data-urlencode "limit=${LIMIT}" \
    --data-urlencode "event_type=${EVENT_TYPE}" \
    --data-urlencode "after=${after}"
}

download_log() {
  local id="$1"
  echo "ID: ${id} のログを取得中" >&2
  perform_curl "ログのダウンロード id=${id}" \
    -G -L \
    "${API_BASE}/${SCOPE_SEGMENT}/${PRINCIPAL_ID}/logs/${id}" \
    "${AUTH_HEADER[@]}"
}

to_local_human() {
  local iso="$1"
  if [[ -z "${iso}" || "${iso}" == "null" ]]; then
    echo ""
    return 0
  fi

  local iso_norm
  iso_norm=$(echo -n "${iso}" \
    | sed -E 's/\.[0-9]+(Z|[+-][0-9:]+)$/\1/' \
    | sed -E 's/([+-]00:00)$/Z/')

  # macOS/BSD date: UTCをエポック時間に解析してからローカルタイムゾーンでフォーマット
  local epoch
  epoch=$(date -j -u -f "%Y-%m-%dT%H:%M:%SZ" "${iso_norm}" +%s 2>/dev/null) || true
  if [[ -n "${epoch}" ]]; then
    date -r "${epoch}" "+%Y-%m-%d %H:%M:%S %Z" 2>/dev/null && return 0
  fi

  # 解析に失敗した場合は元の値にフォールバック
  echo "${iso}"
}

current_after="${INITIAL_AFTER}"
page=1
total_downloaded=0
while true; do
  echo "ページ ${page} を取得中 after='${current_after}' (ローカル時刻: $(to_local_human "${current_after}"))" >&2
  response_json="$(list_logs "${current_after}")"

  # 現在のページから各IDをカウントしてダウンロード（存在する場合）
  page_count="$(echo "${response_json}" | jq '.data | length')"
  if [[ "${page_count}" -gt 0 ]]; then
    echo "${response_json}" | jq -r '.data[].id' | while read -r id; do
      download_log "${id}"
    done
    total_downloaded=$((total_downloaded + page_count))
  fi

  has_more="$(echo "${response_json}" | jq -r '.has_more')"
  current_after="$(echo "${response_json}" | jq -r '.last_end_time')"
  if [[ "${has_more}" == "true" ]]; then
    page=$((page + 1))
  else
    break
  fi
done

if [[ "${total_downloaded}" -eq 0 && ( -z "${current_after}" || "${current_after}" == "null" ) ]]; then
  echo "event_type ${EVENT_TYPE} で ${INITIAL_AFTER} 以降の結果が見つかりませんでした" >&2
else
  echo "${total_downloaded} 個のログファイルのダウンロードが完了しました（${current_after} まで、ローカル時刻: $(to_local_human "${current_after}")）" >&2
fi
```

## オプション2: Windows ベース

前提条件:
- スクリプトを `download_compliance_files.ps1` としてローカルに保存する
- PowerShell（バージョン5.1以上）を開き、スクリプトが保存されているディレクトリに移動する

`.\download_compliance_files.ps1 <workspace_or_org_id> <event_type> <limit> <after>` のようにスクリプトを実行する

```ps
#!/usr/bin/env pwsh
#Requires -Version 5.1

Set-StrictMode -Version Latest
$ErrorActionPreference = 'Stop'

Add-Type -AssemblyName System.Web

function Show-Usage {
    [Console]::Error.WriteLine(@"
使用方法: .\download_compliance_files.ps1 <workspace_or_org_id> <event_type> <limit> <after>

例:
  `$env:COMPLIANCE_API_KEY = '<KEY>'
  .\download_compliance_files.ps1 f7f33107-5fb9-4ee1-8922-3eae76b5b5a0 AUTH_LOG 100 (Get-Date -AsUTC).AddDays(-1).ToString('yyyy-MM-ddTHH:mm:ssZ') |
    Out-File -Encoding utf8 output.jsonl

例 (組織ID):
  `$env:COMPLIANCE_API_KEY = '<KEY>'
  .\download_compliance_files.ps1 org-p13k3klgno5cqxbf0q8hpgrk AUTH_LOG 100 (Get-Date -AsUTC).AddDays(-1).ToString('yyyy-MM-ddTHH:mm:ssZ') |
    Out-File -Encoding utf8 output.jsonl
"@)
}

if ($args.Count -ne 4) {
    Show-Usage
    exit 2
}

if (-not $env:COMPLIANCE_API_KEY) {
    [Console]::Error.WriteLine('COMPLIANCE_API_KEY環境変数を設定する必要があります。')
    exit 2
}

$PrincipalId = $args[0]
$EventType = $args[1]
$Limit = $args[2]
$InitialAfter = $args[3]

$ApiBase = 'https://api.chatgpt.com/v1/compliance'

if ($PrincipalId.StartsWith('org-')) {
    $ScopeSegment = 'organizations'
} else {
    $ScopeSegment = 'workspaces'
}

$handler = [System.Net.Http.HttpClientHandler]::new()
$client = [System.Net.Http.HttpClient]::new($handler)
$client.DefaultRequestHeaders.Authorization = New-Object System.Net.Http.Headers.AuthenticationHeaderValue('Bearer', $env:COMPLIANCE_API_KEY)

function Invoke-ComplianceRequest {
    param(
        [Parameter(Mandatory = $true)] [string] $Description,
        [Parameter(Mandatory = $true)] [string] $Path,
        [hashtable] $Query = @{}
    )

    $builder = [System.UriBuilder]::new("$ApiBase/$ScopeSegment/$PrincipalId/$Path")
    $queryString = [System.Web.HttpUtility]::ParseQueryString($builder.Query)
    foreach ($key in $Query.Keys) {
        $queryString[$key] = $Query[$key]
    }
    $builder.Query = $queryString.ToString()

    try {
        $response = $client.GetAsync($builder.Uri).GetAwaiter().GetResult()
    } catch {
        [Console]::Error.WriteLine("$Description中にネットワーク/トランスポートエラーが発生しました")
        exit 1
    }

    $body = $response.Content.ReadAsStringAsync().GetAwaiter().GetResult()
    if (-not $response.IsSuccessStatusCode) {
        [Console]::Error.WriteLine("${Description}中にHTTPエラー $($response.StatusCode.value__) が発生しました:")
        if ($body) {
            try {
                $parsed = $body | ConvertFrom-Json
                $parsed | ConvertTo-Json -Depth 10 | Write-Error
            } catch {
                [Console]::Error.WriteLine($body)
            }
        }
        exit 1
    }

    Write-Output $body
}

function List-Logs {
    param(
        [Parameter(Mandatory = $true)] [string] $After
    )

    Invoke-ComplianceRequest -Description "ログの一覧取得 (after=$After, event_type=$EventType, limit=$Limit)" -Path 'logs' -Query @{
        limit      = $Limit
        event_type = $EventType
        after      = $After
    }
}

function Download-Log {
    param(
        [Parameter(Mandatory = $true)] [string] $Id
    )

    [Console]::Error.WriteLine("ID: $Id のログを取得中")
    Invoke-ComplianceRequest -Description "ログのダウンロード id=$Id" -Path "logs/$Id"
}

function ConvertTo-LocalHuman {
    param(
        [string] $Iso
    )

    if (-not $Iso -or $Iso -eq 'null') {
        return ''
    }

    try {
        $dt = [datetimeoffset]::Parse($Iso)
        return $dt.ToLocalTime().ToString('yyyy-MM-dd HH:mm:ss zzz')
    } catch {
        return $Iso
    }
}

$currentAfter = $InitialAfter
$page = 1
$totalDownloaded = 0
while ($true) {
    [Console]::Error.WriteLine("ページ $page を取得中 after='$currentAfter' (ローカル時刻: $(ConvertTo-LocalHuman -Iso $currentAfter))")
    $responseJson = List-Logs -After $currentAfter
    $responseObj = $responseJson | ConvertFrom-Json

    $pageCount = $responseObj.data.Count
    if ($pageCount -gt 0) {
        foreach ($entry in $responseObj.data) {
            Download-Log -Id $entry.id
        }
        $totalDownloaded += $pageCount
    }

    $hasMore = $false
    if ($null -ne $responseObj.has_more) {
        $hasMore = [System.Convert]::ToBoolean($responseObj.has_more)
    }

    $currentAfter = $responseObj.last_end_time
    if ($hasMore) {
        $page += 1
    } else {
        break
    }
}

if ($totalDownloaded -eq 0 -and ([string]::IsNullOrEmpty($currentAfter) -or $currentAfter -eq 'null')) {
    [Console]::Error.WriteLine("$InitialAfter 以降のevent_type $EventType に対する結果が見つかりませんでした")
} else {
    [Console]::Error.WriteLine("$currentAfter まで $totalDownloaded 個のログファイルのダウンロードが完了しました (ローカル時刻: $(ConvertTo-LocalHuman -Iso $currentAfter))")
}

$client.Dispose()
$handler.Dispose()
```